In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as k
from  tensorflow.keras import layers,Sequential

In [19]:
x = tf.range(10)
x = tf.random.shuffle(x)
net = layers.Embedding(10, 4)
out = net(x)
print(x)
print(out)

tf.Tensor([3 5 8 1 6 9 4 7 2 0], shape=(10,), dtype=int32)
tf.Tensor(
[[ 0.01426549  0.01124696  0.02602378  0.04556921]
 [-0.04793354 -0.01993302 -0.01701725 -0.03010699]
 [ 0.03379477  0.0349181  -0.0118949  -0.02840271]
 [-0.02403321 -0.00167481 -0.00572225  0.01567317]
 [ 0.00122722  0.03358312  0.0104417   0.00359457]
 [-0.0042655  -0.02397506  0.03648127 -0.00220952]
 [ 0.04441205  0.02992076  0.01841389 -0.03264745]
 [ 0.02902984  0.0234506  -0.03512247 -0.04139868]
 [ 0.00970163  0.00349671  0.04922913 -0.0227282 ]
 [ 0.03482169 -0.0440794  -0.00169345  0.04115376]], shape=(10, 4), dtype=float32)
